In [1]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession

from sedona.utils.adapter import Adapter
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.core.formatMapper.shapefileParser import ShapefileReader

In [2]:
spark = SparkSession. \
    builder. \
    appName('appName'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.datasyslab:geotools-wrapper:geotools-24.0'). \
    getOrCreate()

In [3]:
SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext

In [4]:
os.getcwd()

'/home/jovyan/work/notebooks'

In [5]:
point_csv_df = spark.read.format("csv").\
    option("delimiter", ",").\
    option("header", "true").\
    load("../data/sample_points.csv")

point_csv_df.head()
temptablename = "gpstracks"

In [6]:
point_csv_df.createOrReplaceTempView(temptablename)

point_df = spark.sql("select ST_Point(cast({:}.longitude as Decimal(24,20)), cast({:}.latitude as Decimal(24,20))) as gps from {:}".format(*[temptablename]*3))
point_df.show(5)

+--------------------+
|                 gps|
+--------------------+
|POINT (-43.307411...|
|POINT (-43.186813...|
|POINT (-43.575729...|
|POINT (-43.274319...|
|POINT (-43.2356 -...|
+--------------------+
only showing top 5 rows



In [ ]:
aois = ShapefileReader.readToGeometryRDD(sc, "/home/jovyan/work/data/sample_polygons/")
aois_df = Adapter.toDf(aois, spark)
aois_df.createOrReplaceTempView("aois")
aois_df.printSchema()

In [ ]:
aois.boundary()